In [1]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore")
import sys
sys.path.append("/home/lcz/lenlp/text-generation/py_project")

## 配置GPU使用

In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

## 加载数据集

In [3]:
from utils.data_loader import load_dataset, load_test_dataset
from utils.linux_config import embedding_matrix_path
from utils.wv_loader import load_embedding_matrix, load_vocab

root path: /home/lcz/lenlp/text-generation


In [4]:
train_X, train_Y, test_X = load_dataset()

In [5]:
embedding_matrix = load_embedding_matrix("./../data/embedding_matrix")

In [6]:
vocab_path = "./../data/vocab.txt"
vocab, reverse_vocab = load_vocab(vocab_path)

## 配置参数

In [7]:
units = 1024
params = {}
params["vocab_size"] = len(vocab)
params["embed_size"] = 300
params["enc_units"] = units
params["attn_units"] = units
params["dec_units"] = units
params["batch_size"] = 64
params["epochs"] = 2
params["max_enc_len"] = 200
params["max_dec_len"] = 41

## 构建数据集

In [8]:
import tensorflow as tf
steps_per_epoch = len(train_X) // params["batch_size"]
dataset = tf.data.Dataset.from_tensor_slices((train_X, train_Y)).shuffle(params["batch_size"])
dataset = dataset.batch(params["batch_size"], drop_remainder=True)

## 构建模型

In [9]:
from seq2seq.layers import Encoder, BahdanauAttention, Decoder

root path: /home/lcz/lenlp/text-generation


### Encoder

In [10]:
encoder = Encoder(params["vocab_size"], params["embed_size"], embedding_matrix, params["enc_units"], params["batch_size"])
enc_hidden = encoder.initialize_hidden_state()
example_input_batch = tf.ones(shape=(params["batch_size"], params["max_enc_len"]), dtype=tf.int32)
sample_output, sample_hidden = encoder(example_input_batch, enc_hidden)
sample_output.shape
sample_hidden.shape

(64, 200, 300)


TensorShape([64, 1024])

### Decoder

In [11]:
decoder = Decoder(params["vocab_size"], params["embed_size"], embedding_matrix, params["enc_units"], params["batch_size"])
sample_decoder_output, _, _ = decoder(tf.random.uniform((64, 1)), sample_hidden, sample_output)
sample_decoder_output.shape

TensorShape([64, 32909])

## 保存点设置

In [12]:
optimizer = tf.keras.optimizers.Adam(name='Adam',learning_rate=0.001)
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')


pad_index=vocab['<PAD>']
nuk_index=vocab['<UNK>']

def loss_function(real, pred):
    pad_mask = tf.math.equal(real, pad_index)
    nuk_mask = tf.math.equal(real, nuk_index)
    mask = tf.math.logical_not(tf.math.logical_or(pad_mask,nuk_mask))
    
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

checkpoint_dir = "./../data/checkpoints/beam_search_training_checkpoints_mask_loss_dim300_seq"
import os
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer, encoder=encoder, decoder=decoder)

## 训练模型

In [13]:
@tf.function
def train_step(inp, targ, enc_hidden):
    loss = 0
    
    with tf.GradientTape() as tape:
        # 1. 构建encoder
        enc_output, enc_hidden = encoder(inp, enc_hidden)
        # 2. 复制
        dec_hidden = enc_hidden
        # 3. <START> * BATCH_SIZE 
        dec_input = tf.expand_dims([vocab['<START>']] * params["batch_size"], 1)
        
        # Teacher forcing - feeding the target as the next input
        for t in range(1, targ.shape[1]):
            # decoder(x, hidden, enc_output)
            predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
            
            loss += loss_function(targ[:, t], predictions)

            # using teacher forcing
            dec_input = tf.expand_dims(targ[:, t], 1)

        batch_loss = (loss / int(targ.shape[1]))

        variables = encoder.trainable_variables + decoder.trainable_variables

        gradients = tape.gradient(loss, variables)

        optimizer.apply_gradients(zip(gradients, variables))

        return batch_loss

In [ ]:
import time
epochs = params["epochs"]
# 如果检查点存在，则恢复最新的检查点。
if checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir)):
    checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))
    print ('Latest checkpoint restored!!')
    
for epoch in range(epochs):
    start = time.time()
    total_loss = 0
    enc_hidden = encoder.initialize_hidden_state()

    for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
        batch_loss = train_step(inp, targ, enc_hidden)
        total_loss += batch_loss

        if batch % 1 == 0:
            print('Epoch {} Step {} Loss {:.4f}'.format(epoch + 1,
                                                         batch,
                                                         batch_loss.numpy()))
    # saving (checkpoint) the model every 2 epochs
    if (epoch + 1) % 2 == 0:
        checkpoint.save(file_prefix = checkpoint_prefix)
        print ('Saving checkpoint for epoch {} at {}'.format(epoch+1,
                                                             checkpoint_prefix))

    print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Latest checkpoint restored!!
(64, 260, 300)
(64, 260, 300)
Epoch 1 Step 0 Loss 5.0501
Epoch 1 Step 1 Loss 5.7422
Epoch 1 Step 2 Loss 5.4559
Epoch 1 Step 3 Loss 4.3635
Epoch 1 Step 4 Loss 4.4234
Epoch 1 Step 5 Loss 3.4175
Epoch 1 Step 6 Loss 2.3852
Epoch 1 Step 7 Loss 2.0323
Epoch 1 Step 8 Loss 2.3369
Epoch 1 Step 9 Loss 2.7592
Epoch 1 Step 10 Loss 3.6975
Epoch 1 Step 11 Loss 3.2331
Epoch 1 Step 12 Loss 2.5499
Epoch 1 Step 13 Loss 3.0730
Epoch 1 Step 14 Loss 3.4795
Epoch 1 Step 15 Loss 3.6937
Epoch 1 Step 16 Loss 2.5997
Epoch 1 Step 17 Loss 1.8833
Epoch 1 Step 18 Loss 1.8907
Epoch 1 Step 19 Loss 1.6549
Epoch 1 Step 20 Loss 2.1969
Epoch 1 Step 21 Loss 2.3029
Epoch 1 Step 22 Loss 2.4705
Epoch 1 Step 23 Loss 2.7176
Epoch 1 Step 24 Loss 2.5495
Epoch 1 Step 25 Loss 2.3808
Epoch 1 Step 26 Loss 3.4386
Epoch 1 Step 27 Loss 3.0722
Epoch 1 Step 28 Loss 3.4121
Epoch 1 Step 29 Loss 3.0982
Epoch 1 Step 30 Loss 2.7119
Epoch 1 Step 31 Loss 2.6307
Epoch 1 Step 32 Loss 2.7875
Epoch 1 Step 33 Loss 2.3432

## beam search

In [ ]:
class Hypothesis:
    """ Class designed to hold hypothesises throughout the beamSearch decoding """

    def __init__(self, tokens, log_probs, hidden, attn_dists):
        self.tokens = tokens  # list of all the tokens from time 0 to the current time step t
        self.log_probs = log_probs  # list of the log probabilities of the tokens of the tokens
        self.hidden = hidden  # decoder hidden state after the last token decoding
        self.attn_dists = attn_dists  # attention dists of all the tokens
        self.abstract = ""

    def extend(self, token, log_prob, hidden, attn_dist):
        """Method to extend the current hypothesis by adding the next decoded token and all the informations associated with it"""
        return Hypothesis(tokens=self.tokens + [token],  # we add the decoded token
                          log_probs=self.log_probs + [log_prob],  # we add the log prob of the decoded token
                          hidden=hidden,  # we update the state
                          attn_dists=self.attn_dists + [attn_dist])

    @property
    def latest_token(self):
        return self.tokens[-1]

    @property
    def tot_log_prob(self):
        return sum(self.log_probs)

    @property
    def avg_log_prob(self):
        return self.tot_log_prob / len(self.tokens)


def beam_decode(model, batch, vocab, params):
    # 初始化mask
    start_index = vocab.word_to_id(vocab.START_DECODING)
    stop_index = vocab.word_to_id(vocab.STOP_DECODING)
    unk_index = vocab.word_to_id(vocab.UNKNOWN_TOKEN)

    batch_size = params['batch_size']

    # 单步decoder
    def decoder_onestep(enc_output, dec_input, dec_hidden, enc_extended_inp, batch_oov_len):
        # 单个时间步 运行
        # dec_input, dec_hidden, enc_output, enc_extended_inp, batch_oov_len
        final_preds, dec_hidden, context_vector, attention_weights, p_gens = model.call_decoder_onestep(dec_input,
                                                                                                        dec_hidden,
                                                                                                        enc_output,
                                                                                                        enc_extended_inp,
                                                                                                        batch_oov_len)
        # 拿到top k个index 和 概率
        top_k_probs, top_k_ids = tf.nn.top_k(tf.squeeze(final_preds), k=params["beam_size"] * 2)
        # 计算log概率
        top_k_log_probs = tf.math.log(top_k_probs)

        results = {
            # 'final_dists': preds,
            "last_context_vector": context_vector,
            "dec_hidden": dec_hidden,
            "attention_weights": attention_weights,
            "top_k_ids": top_k_ids,
            "top_k_log_probs": top_k_log_probs,
            "p_gen": p_gens}

        # 返回需要保存的中间结果和概率
        return results

    # 测试数据的输入
    enc_input = batch[0]["enc_input"]

    # 计算第encoder的输出
    enc_output, enc_hidden = model.call_encoder(enc_input)

    # 初始化batch size个 假设对象
    hyps = [Hypothesis(tokens=[start_index],
                       log_probs=[0.0],
                       hidden=enc_hidden[0],
                       attn_dists=[],
                       ) for _ in range(batch_size)]
    # 初始化结果集
    results = []  # list to hold the top beam_size hypothesises
    # 遍历步数
    steps = 0  # initial step

    enc_extended_inp = batch[0]["extended_enc_input"]
    batch_oov_len = batch[0]["max_oov_len"]

    # 长度还不够 并且 结果还不够 继续搜索
    while steps < params['max_dec_len'] and len(results) < params['beam_size']:
        # 获取最新待使用的token
        latest_tokens = [h.latest_token for h in hyps]
        # 替换掉 oov token unknown token
        latest_tokens = [t if t in vocab.id2word else unk_index for t in latest_tokens]
        # 获取所以隐藏层状态
        hiddens = [h.hidden for h in hyps]

        dec_input = tf.expand_dims(latest_tokens, axis=1)
        dec_hidden = tf.stack(hiddens, axis=0)

        # 单步运行decoder 计算需要的值
        decoder_results = decoder_onestep(enc_output,
                                          dec_input,
                                          dec_hidden,
                                          enc_extended_inp,
                                          batch_oov_len)

        # preds = decoder_results['final_dists']
        # context_vector = decoder_results['last_context_vector']

        dec_hidden = decoder_results['dec_hidden']
        attention_weights = decoder_results['attention_weights']
        top_k_log_probs = decoder_results['top_k_log_probs']
        top_k_ids = decoder_results['top_k_ids']

        # print('top_k_ids {}'.format(top_k_ids))

        # 现阶段全部可能情况
        all_hyps = []
        # 原有的可能情况数量
        num_orig_hyps = 1 if steps == 0 else len(hyps)

        # 遍历添加所有可能结果
        for i in range(num_orig_hyps):
            h, new_hidden, attn_dist = hyps[i], dec_hidden[i], attention_weights[i]
            # 分裂 添加 beam size 种可能性
            for j in range(params['beam_size'] * 2):
                # 构造可能的情况
                new_hyp = h.extend(token=top_k_ids[i, j].numpy(),
                                   log_prob=top_k_log_probs[i, j],
                                   hidden=new_hidden,
                                   attn_dist=attn_dist)
                # 添加可能情况
                all_hyps.append(new_hyp)

        # 重置
        hyps = []
        # 按照概率来排序
        sorted_hyps = sorted(all_hyps, key=lambda h: h.avg_log_prob, reverse=True)

        # 筛选top前beam_size句话
        for h in sorted_hyps:
            if h.latest_token == stop_index:
                # 长度符合预期,遇到句尾,添加到结果集
                if steps >= params['min_dec_steps']:
                    results.append(h)
            else:
                # 未到结束 ,添加到假设集
                hyps.append(h)

            # 如果假设句子正好等于beam_size 或者结果集正好等于beam_size 就不在添加
            if len(hyps) == params['beam_size'] or len(results) == params['beam_size']:
                break

        steps += 1

    if len(results) == 0:
        results = hyps

    hyps_sorted = sorted(results, key=lambda h: h.avg_log_prob, reverse=True)
    best_hyp = hyps_sorted[0]
    best_hyp.abstract = " ".join([vocab.id_to_word(index) for index in best_hyp.tokens])
    best_hyp.text = batch[0]["article"].numpy()[0].decode()
    
    return best_hyp